In [18]:
import os,shutil,importlib
import Development as dev
importlib.reload(dev)

projectPath = os.path.join(os.getcwd(),'exampleProject')
if os.path.isdir(projectPath):
    shutil.rmtree(projectPath)


rd = dev.Parse(rootPath=projectPath,
                siteID = 'FI',
                # importRoot = r'E:\highfreq\FI_Raw_EC_Data'
                importRoot = os.path.join(os.getcwd(),'exampleInputs'),
                importRelative = os.path.join('GSC_EC'),
                includeTag = ['Flux'],
                )



Consider fixing to allow multi-site imports with on call?
{'TOB3\\Flux_Data': {'Flux_Data361.dat': {'groupID': '2024-07-23T0230', 'dataSource': ['GSC_EC', '20240724', 'Flux_Data361.dat']}, 'Flux_Data362.dat': {'groupID': '2024-07-23T0230', 'dataSource': ['GSC_EC', '20240724', 'Flux_Data362.dat']}, 'Flux_Data432.dat': {'groupID': '2024-07-23T0230', 'dataSource': ['GSC_EC', '20240724', 'Flux_Data432.dat']}, 'Flux_Data433.dat': {'groupID': '2024-07-23T0230', 'dataSource': ['GSC_EC', '20240724', 'Flux_Data433.dat']}, 'Flux_Data462.dat': {'groupID': '2024-07-25T0430', 'dataSource': ['GSC_EC', '20240914', 'Flux_Data462.dat']}, 'Flux_Data1807.dat': {'groupID': '2024-08-22T0300', 'dataSource': ['GSC_EC', '20240914', 'Flux_Data1807.dat']}, 'Flux_Data1808.dat': {'groupID': '2024-08-22T0300', 'dataSource': ['GSC_EC', '20240914', 'Flux_Data1808.dat']}, 'Flux_Data1809.dat': {'groupID': '2024-08-22T0300', 'dataSource': ['GSC_EC', '20240914', 'Flux_Data1809.dat']}, 'Flux_Data1810.dat': {'groupID': '2

In [55]:
importlib.reload(dev)

rd = dev.Parse(mode = 'sync',
            rootPath=projectPath,
            Verbose = True,
            siteID = 'FI',)


Accepting changes in  sourceInfo\Frequency  from  1.0s  to  0.1s
Accepting changes in  fileContents\metadata\Timing\acquisition_frequency  from  1.0  to  10.0
Accepting changes in  fileContents\metadata\Instruments\instr_1_height  from  3  to  .21
Accepting changes in  fileContents\metadata\Instruments\instr_1_north_offset  from  0  to  35
Accepting changes in  fileContents\metadata\Instruments\instr_2_tube_length  from  33  to  15
Accepting changes in  sourceInfo\Frequency  from  0.1s  to  0.05s
Accepting changes in  fileContents\metadata\Timing\acquisition_frequency  from  10.0  to  20.0


In [196]:
from dataclasses import dataclass,field

@dataclass
class firstStageTemplate:
    input_name: str = None
    ignore: bool = False
    standard_name: str = None
    unit_in: str = None
    unit_out: str = None
    conversion: str = None
    sensor: str = None
    dataType: str = None

@dataclass
class importTraces:
    input_name: str = None
    output_name: str = None
    sensor: str = None
    unit_in: str = None
    dataType: str = None
    ignore: bool = False

@dataclass
class Processing(importTraces):
    inputMetadata: dict = field(default_factory=dict)
    position: list = field(default_factory=lambda:[1,1,1])
    dateRange: list = field(default_factory=list)

    def __post_init__(self):
        self.firstStage = {
            record:importTraces(**{k:v for k,v in entry.items() if k in importTraces().__dict__.keys()}).__dict__
            for record,entry in self.inputMetadata.items()}
        
    
item = rd.Metadata['HOBO-readout_SHSCWSMSGT']['2024-07-20T0000_20750527_20750527-SHSCWSMSGT240720_1800s_GMT+0000']
fS = Processing(inputMetadata=item)


In [198]:
import helperFunctions as helper
helper.unpackDict(rd.Metadata,limit=1)

{'HOBO-readout_SHSCWSMSGT\\2024-07-20T0000_20750527_20750527-SHSCWSMSGT240720_1800s_GMT+0000': {'RecordNumber': {'unit_in': '',
   'logger': '',
   'sensor': '',
   'ignore': True,
   'dataType': 'int32'},
  'Date_Time': {'unit_in': ' GMT+00:00',
   'logger': '',
   'sensor': '',
   'ignore': True,
   'dataType': 'object'},
  'Temp_LBL_5': {'unit_in': ' °C ',
   'logger': 'LGR S/N: 20750527',
   'sensor': ' SEN S/N: 20750527',
   'ignore': False,
   'dataType': 'float32'},
  'Temp_LBL_25': {'unit_in': ' °C ',
   'logger': 'LGR S/N: 20750527',
   'sensor': ' SEN S/N: 20750527',
   'ignore': False,
   'dataType': 'float32'},
  'Temp_LBL_50': {'unit_in': ' °C ',
   'logger': 'LGR S/N: 20750527',
   'sensor': ' SEN S/N: 20750527',
   'ignore': False,
   'dataType': 'float32'},
  'Temp_LBL_100': {'unit_in': ' °C ',
   'logger': 'LGR S/N: 20750527',
   'sensor': ' SEN S/N: 20750527',
   'ignore': False,
   'dataType': 'float32'},
  'Host_Connected': {'unit_in': 'LGR S/N: 20750527',
   'logge

In [261]:
from dataclasses import dataclass,field

@dataclass
class firstStage():
    raw_name: str = ''
    ignore: bool = False
    standard_name: str = None
    unit_in: str = None
    unit_out: str = None
    conversion: str = None
    sensor: str = None
    dataType: str = None
    position: list = field(default_factory=lambda:[1,1,1])
    dateRange: list = field(default_factory=list)
    entry: dict = field(default_factory=dict)

    def __post_init__(self):
        if self.standard_name is None:
            self.standard_name = self.raw_name
        self.standard_name = self.standard_name+'_' +'_'.join([str(p) for p in self.position])
        if self.unit_out is None:
            self.unit_out = self.unit_in
        self.entry = {self.raw_name:{
            'AmerifluxName':self.standard_name,
            'ignore':self.ignore,
            'unit_in':self.unit_in,
            'unit_out':self.unit_out,
            'dataType':self.dataType,
            'sensor':self.sensor,
            'dateRange':self.dateRange,
            'comments':''
        }}
    
firstStage()


firstStage(raw_name='', ignore=False, standard_name='_1_1_1', unit_in=None, unit_out=None, conversion=None, sensor=None, dataType=None, position=[1, 1, 1], dateRange=[], entry={'': {'AmerifluxName': '_1_1_1', 'ignore': False, 'unit_in': None, 'unit_out': None, 'dataType': None, 'sensor': None, 'dateRange': [], 'comments': ''}})

In [159]:
for v in item.values():
    print(v)

{'unit_in': '', 'logger': '', 'sensor': '', 'ignore': True, 'dataType': 'int32'}
{'unit_in': ' GMT+00:00', 'logger': '', 'sensor': '', 'ignore': True, 'dataType': 'object'}
{'unit_in': ' °C ', 'logger': 'LGR S/N: 20750527', 'sensor': ' SEN S/N: 20750527', 'ignore': False, 'dataType': 'float32'}
{'unit_in': ' °C ', 'logger': 'LGR S/N: 20750527', 'sensor': ' SEN S/N: 20750527', 'ignore': False, 'dataType': 'float32'}
{'unit_in': ' °C ', 'logger': 'LGR S/N: 20750527', 'sensor': ' SEN S/N: 20750527', 'ignore': False, 'dataType': 'float32'}
{'unit_in': ' °C ', 'logger': 'LGR S/N: 20750527', 'sensor': ' SEN S/N: 20750527', 'ignore': False, 'dataType': 'float32'}
{'unit_in': 'LGR S/N: 20750527', 'logger': '', 'sensor': '', 'ignore': True, 'dataType': 'object'}
{'unit_in': 'LGR S/N: 20750527', 'logger': '', 'sensor': '', 'ignore': True, 'dataType': 'object'}


In [149]:
import yaml
fullFirstStage = {}
for table in rd.Metadata.keys():
    for ID,item in rd.Metadata[table].items():
        print(ID)
        # fullFirstStage[table] = {}
        # for raw_name,variable in item.items():
        #     fS = firstStage(raw_name=raw_name,
        #                **{k:v for k,v in variable.items() if k in firstStage().__dict__.keys()}
        #                )
        #     fullFirstStage[table][raw_name] = fS.entry

print(yaml.dump(fullFirstStage,sort_keys=False))

2024-07-20T0000_20750527_20750527-SHSCWSMSGT240720_1800s_GMT+0000
2024-07-20T0000_20750528_20750528-SHSCSSMSGT240720_1800s_GMT+0000
2024-07-23T0230_57840_SCL_2024_07_20_005s_CR1000X
2024-07-25T0430_57840_SCL_2024_07_20_005s_CR1000X
2024-08-22T0300_57840_SCL_2024_07_25_005s_CR1000X
2024-07-20T2101_57840_SCL_2024_07_20_864000s_CR1000X
2024-07-20T2321_57840_SCL_2024_07_20_864000s_CR1000X
2024-07-23T0000_57840_SCL_2024_07_20_18000s_CR1000X
2024-07-25T0000_57840_SCL_2024_07_20_18000s_CR1000X
2024-09-13T1648_OverWinterDEF_18000s_CR10X
{}



In [39]:
# firstStage('a')

# for table in rd.Metadata.keys():
#     for ID,item in rd.Metadata[table].items():
#         print(table)
#         print(item)


    # def firstStage(self,md):
    #     processing = {}
    #     template = self.projectConfig['Database']['configFiles']['firstStage']['defaultVariable']
    #     for name,item in md['fileContents'].items():
    #         processing[name] = copy.deepcopy(template)
    #         if md['Type'] != 'MixedArray':
    #             for key in template.keys():
    #                 if key == 'date_range':
    #                     processing[name][key] = [md['Timestamp']]
    #                 elif key in item.keys():
    #                     processing[name][key] = [item[key]]
    #                 else:
    #                     processing[name][key] = [template[key]]
    #         else:
    #             for ar,it in item.items():
    #                 for key in template.keys():
    #                     if key == 'date_range':
    #                         processing[name][key] = [md['Timestamp']]
    #                     elif key in item.keys():
    #                         processing[name][key] = [it[key]]
    #                     else:
    #                         processing[name][key] = [template[key]]
    #     return(processing)

firstStage(standard_name='a')

In [23]:
# rd.fileList['TOB3_Flux_Data']


{'2024-07-23T0230_57840_SCL_2024_07_20_005s_CR1000X': [['GSC_EC',
   '20240724',
   'Flux_Data361.dat'],
  ['GSC_EC', '20240724', 'Flux_Data362.dat'],
  ['GSC_EC', '20240724', 'Flux_Data432.dat'],
  ['GSC_EC', '20240724', 'Flux_Data433.dat'],
  ['GSC_EC', '20240724', 'Flux_Data462.dat']],
 '2024-08-22T0300_57840_SCL_2024_07_25_005s_CR1000X': [['GSC_EC',
   '20240914',
   'Flux_Data1807.dat'],
  ['GSC_EC', '20240914', 'Flux_Data1808.dat'],
  ['GSC_EC', '20240914', 'Flux_Data1809.dat'],
  ['GSC_EC', '20240914', 'Flux_Data1810.dat']]}

In [112]:
importlib.reload(dev)

dev.processing(siteID='a')

In [7]:
# import datetime
# datetime.datetime.now().year
# p.database.map()
for key,value in None:
    pass

TypeError: 'NoneType' object is not iterable

In [ ]:
import helperFunctions as h

for k(h.unpackDict('1'))

NoneType

In [133]:
# rd.project['rawData']
rd.getMetadata()

Consider fixing to allow multi-site imports with on call?


In [134]:
rd.project['rawData']

{'SCL': {'TOB3_Flux_Data': {'2024-07-23T0230_57840_SCL_2024_07_20_005s_CR1000X': [['GSC_EC',
     '20240724',
     'Flux_Data361.dat'],
    ['GSC_EC', '20240724', 'Flux_Data362.dat'],
    ['GSC_EC', '20240724', 'Flux_Data432.dat'],
    ['GSC_EC', '20240724', 'Flux_Data433.dat']],
   '2024-07-25T0430_57840_SCL_2024_07_20_005s_CR1000X': [['GSC_EC',
     '20240724',
     'Flux_Data462.dat']],
   '2024-08-22T0300_57840_SCL_2024_07_25_005s_CR1000X': [['GSC_EC',
     '20240914',
     'Flux_Data1807.dat'],
    ['GSC_EC', '20240914', 'Flux_Data1808.dat'],
    ['GSC_EC', '20240914', 'Flux_Data1809.dat'],
    ['GSC_EC', '20240914', 'Flux_Data1810.dat']]}}}

In [ ]:

# mp = dev.importRawData(projectPath=projectPath,
#                  siteID = 'SCL',
#                 importRoot = os.path.join(os.getcwd(),'exampleInputs'),
#                 importRelative = os.path.join('GSC_EC'),
#                 includeTag = ['2024','Flux'],
#                 # fileType = 'dat'
#                 )
# 25T0430_57840_SCL_2024_07_20_005s_CR1000X
# 25T0430_57840_SCL_2024_07_20_005s_CR1000X

# mp = dev.importRawData(projectPath=projectPath,
#                  siteID = 'SCL',
#                 importRoot = os.path.join(os.getcwd(),'exampleInputs'),
#                 importRelative = os.path.join('FI_EC'),
#                 includeTag = ['2017'],
#                 fileType = 'GHG'
#                 )

{'TOB3_Flux_Data': {'2024-07-23T0230_57840_SCL_2024_07_20_005s_CR1000X': None,
  '2024-07-25T0430_57840_SCL_2024_07_20_005s_CR1000X': {'values_changed': {'CO2': {'unit_in': {'new_value': 'mg/mol',
      'old_value': 'mmol/m^3',
      'acceptNew': True}},
    'H2O': {'unit_in': {'new_value': 'g/mol',
      'old_value': 'mmol/m^3',
      'acceptNew': True}}}},
  '2024-08-22T0300_57840_SCL_2024_07_25_005s_CR1000X': {'values_changed': {'Uz': {'unit_in': {'new_value': 'm/s',
      'old_value': 'mm/s',
      'acceptNew': True}},
    'CO2': {'unit_in': {'new_value': 'mmol/m^3',
      'old_value': 'mg/mol',
      'acceptNew': True}},
    'H2O': {'unit_in': {'new_value': 'mmol/m^3',
      'old_value': 'g/mol',
      'acceptNew': True}}}}},
 'TOB3_MetaData': {'2024-07-20T2101_57840_SCL_2024_07_20_864000s_CR1000X': None,
  '2024-07-20T2321_57840_SCL_2024_07_20_864000s_CR1000X': {'dictionary_item_added': {'Zm_adj_Avg': {'unit_in': 'm',
     'operation': 'Avg',
     'dataType': 'float32',
     'ign

In [81]:
from dataclasses import dataclass
@dataclass(kw_only=True)
class GrandParent:
    value_for_grandparent: int = 1

@dataclass(kw_only=True)
class Parent(GrandParent):
    value_for_parent: int = 1

# @dataclass(kw_only=True)
class Child(Parent):
    
    def __init__(self,value_for_child,**kwds):
        self.value_for_child = value_for_child
        super(Parent, self).__init__(**kwds)

# Example usage
c = Child(value_for_child=10)#, value_for_parent=20, value_for_grandparent=30)
print(c)

Child(value_for_grandparent=1, value_for_parent=1)


In [ ]:
import os
from dataclasses import dataclass,field


@dataclass(kw_only=True)
class Second:
  a: int = 1
  def __post_init__(self):
    print ("Second(): entering")
    # super().__post_init__()  
    # super(Second, self).__init__()
    print ("Second(): exiting")

@dataclass(kw_only=True)
class Third(Second):
  b: int = 1
  def __post_init__(self):
    print ("Third(): entering")
    print(self.b)
    super().__post_init__()  
    print ("Third(): exiting")
        

class Project(Third):
  def __init__(self,**kwds):
    
    # projectPath: str = os.getcwd()
    # siteID: list = field(default_factory=list)
    
       
    # def __post_init__(self):
    #     print ("Project(): entering")
    #     for key,value in self.__dataclass_fields__.items():
    #         if value.type == list and not isinstance(self.__dict__[key],value.type):
    #             self.__dict__[key] = list([self.__dict__[key]])
    #     print ("Project(): exiting")
    #     super().__post_init__()  



class myProject(Project,Third,Second):
  def __init__(self,**kwds):
    print ("Fourth(): entering")
    super(myProject, self).__init__(**kwds)
    print ("Fourth(): exiting")
# class Alt(Third):
#     def __init__(self):
#         super(Alt, self).__init__()
#         print("Alt")

myProject(projectPath = 'c',siteID='A',b=2)

Fourth(): entering
Project(): entering
Project(): exiting
Third(): entering
2
Second(): entering
Second(): exiting
Third(): exiting
Fourth(): exiting


myProject(a=1, b=2, projectPath='c', siteID=['A'])

In [285]:

from dataclasses import dataclass,field

@dataclass
class rawData():
    # Initializes a raw data import
    importRoot: str = ''
    importRelative: str = ''
    importFileList: list = field(default_factory=list)
    fileType: list = field(default_factory=list)
    includeTag: list = field(default_factory=list)
    excludeTag: list = field(default_factory=list)

@dataclass
class configFiles:
    # This object initializes the projects default settings and loads the current state the state of database
    projectPath: str = os.getcwd()
    # debug: bool = False
    # delimiter: str = '_'
    # _scriptDir = os.path.abspath(os.path.split(__file__)[0])
    siteID: list = field(default_factory=list)
    projectConfig: dict = field(default_factory=dict)
    # _globalConfig: dict = field(init=False)
        

class myProject(configFiles,rawData):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)

myProject()

# for type,values in mp.projectConfig['configFiles']['SCL']['metadata'].items():
#     for item,metadata in values.items():
#         print(item,metadata)

myProject(projectPath='c:\\Users\\User\\GSC_Work\\EC_Processing_Toolkit', siteID=[], projectConfig={})

In [31]:
from dataclasses import dataclass,field

@dataclass
class First:
  a: int = 1
  def __post_init__(self):
    print ("First(): entering")
    # super(First, self).__init__()
    print ("First(): exiting")

# class Second():
#   def __init__(self):
class Second:
  b: int = 1
  def __post_init__(self):
    print ("Second(): entering")
    super().__post_init__()  
    # super(Second, self).__init__()
    print ("Second(): exiting")

class Third:
  b: int = 1
  def __post_init__(self):
    print ("Third(): entering")
    super().__post_init__()  
    print ("Third(): exiting")
        
class Fourth(Third,Second,First):
  def __init__(self,**kwds):
    print ("Fourth(): entering")
    super(Fourth, self).__init__(**kwds)
    print ("Fourth(): exiting")
# class Alt(Third):
#     def __init__(self):
#         super(Alt, self).__init__()
#         print("Alt")

Fourth(projectPath='abc')

Fourth(): entering


TypeError: First.__init__() got an unexpected keyword argument 'projectPath'

In [ ]:
importlib.reload(dev)


if os.path.isdir(projectPath):
    shutil.rmtree(projectPath)

mp = dev.rawData(
  projectPath,
  importRoot = os.path.join(os.getcwd(),'exampleInputs'),
  siteID = 'SCL',
  importPath = os.path.join('GSC_EC'),
  includeTag = ['202407','Flux'],
  fileType = 'dat'
  )


# mp = dev.importRawData(
#   projectPath,
#   importRoot = os.path.join(os.getcwd(),'exampleInputs'),
#   siteID = 'FI',
#   importPath = os.path.join('FI_EC'),
#   includeTag = '2017',
#   fileType = 'ghg'
#   )

In [2]:
import shutil

In [226]:
import os
import re
import sys
import copy
import json
import yaml
import getpass
import datetime
import deepdiff
import numpy as np
import pandas as pd
from dataclasses import dataclass,field

import importlib
from parseGHG import parseGHG
from parseTables import parseTOBA, parseHobo,parseMixedArray
importlib.reload(parseTOBA)
importlib.reload(parseHobo)
importlib.reload(parseMixedArray)
importlib.reload(parseGHG)

import helperFunctions as helper
importlib.reload(helper)

######################
__file__ = os.path.join(os.getcwd(),'xxxx')
######################


@dataclass
class myProject:
    # This object is intended to contain the current state the state of database
    projectPath: str = os.getcwd()
    delimiter: str = '_'
    _scriptDir = os.path.abspath(os.path.split(__file__)[0])
    siteID: list = field(default_factory=list)
    args: dict = field(default_factory=dict)
    projectConfig: dict = field(init=False)
    
    def __post_init__(self):
        if not isinstance(self.__dict__['siteID'],list):
            self.__dict__['siteID'] = list([self.siteID])
        self._globalConfig = self.loadDict(os.path.join(self._scriptDir,'configFiles','projectTemplate.yml'))
        self.projectConfig = self.loadDict(os.path.join(self.projectPath,'configFiles','projectConfig.yml'))
        # self.args = self.defaults | self.args
        # initialize nested dicts that may exist
        if self.projectConfig:
        #     self.makeProject()
        # else:
            for key,value in helper.unpackDict(self.projectConfig).items():
                if type(value) == str:
                    dOut = {key:self.loadDict(os.path.join(self.projectPath,key,value))}
                    if dOut[key] is not None:
                        self.projectConfig = helper.updateDict(self.projectConfig,helper.packDict(dOut),overwrite=True)

    def loadDict(self,file):
        out = None
        if os.path.isfile(file):
            if file.endswith('.yml'):
                with open(file) as f:
                    out = yaml.safe_load(f)
            elif file.endswith('.json'):
                with open(file) as f:
                    out = json.load(f)
        else:
            print(f"{file} does not exist")
        return(out)
    
    def makeProject(self):
        if os.path.isdir(self.projectPath) and len(os.listdir(self.projectPath))>0:
            sys.exit(f'{self.projectPath} is not empty, cannot proceed')
        else:
            self.projectConfig = copy.deepcopy(self._globalConfig)
        
        self.projectConfig = {}
        
    # def
        pattern = rf"__(.+?)__"
        for key,value in helper.unpackDict(self._globalConfig).items():
            key = key + os.path.sep
            matches = re.findall(pattern, key)
            if len(matches)>0:
                for match in matches:
                    attrs = getattr(self,match)
                    for attr in attrs:
                        pth = key.replace(f"__{match}__",attr)
                        self.projectConfig[pth] = value
                        if not os.path.isdir(pth):
                            os.makedirs(pth)
            else:
                self.projectConfig[key] = value
        print(self.projectConfig)
            # print(matches)
            # print(k)
        # rem = rem + [k for k in helper.unpackDict(self._globalConfig,format='~',limit=1) if '~_' in k]
        # print(rem)
        # for key in rem:
        #     key = key.split('~')
        #     if len(key)>1:
        #         self.projectConfig[key[0]].pop(key[1])
        #     else:
        #         self.projectConfig.pop(key[0])
        # self.saveProject()

class configFiles(myProject):
    def __init__(self,**kwds):
        super().__init__(**kwds)

class rawData(configFiles):
    def __init__(self, **kwds):
        super().__init__(**kwds)

projectPath = os.path.join(os.getcwd(),'temp2')
if os.path.isdir(projectPath):
    shutil.rmtree(projectPath)
a = rawData(projectPath=projectPath,siteID='ck')


c:\Users\User\GSC_Work\EC_Processing_Toolkit\temp2\configFiles\projectConfig.yml does not exist


In [139]:
import os
import re
import sys
import copy
import json
import yaml
import getpass
import datetime
import deepdiff
import numpy as np
import pandas as pd
from dataclasses import dataclass,field

import importlib
from parseGHG import parseGHG
from parseTables import parseTOBA, parseHobo,parseMixedArray
importlib.reload(parseTOBA)
importlib.reload(parseHobo)
importlib.reload(parseMixedArray)
importlib.reload(parseGHG)

import helperFunctions as helper
importlib.reload(helper)

######################
__file__ = os.path.join(os.getcwd(),'xxxx')
######################


@dataclass
class myProject:
    # This object is intended to contain the current state the state of database
    projectPath: str = os.getcwd()
    delimiter: str = '_'
    _scriptDir = os.path.abspath(os.path.split(__file__)[0])
    siteID: list = field(default_factory=list)
    args: dict = field(default_factory=dict)
    projectConfig: dict = field(default_factory=dict)
    _globalConfig: dict = field(init=False)
    
    def __post_init__(self):
        if not isinstance(self.__dict__['siteID'],list):
            self.__dict__['siteID'] = list([self.siteID])
        self._globalConfig = self.loadDict(os.path.join(self._scriptDir,'configFiles','projectTemplate.yml'))
        for keys,value in helper.unpackDict(self._globalConfig).items():
            keys = self.subKeys(keys)
            for key in keys:
                if not os.path.isdir(os.path.join(self.projectPath,key)):
                    os.makedirs(os.path.join(self.projectPath,key))
                    value = {}
                elif type(value) is str and os.path.isfile(os.path.join(self.projectPath,key,value)):
                    value = self.loadDict(os.path.join(self.projectPath,key,value))
                self.projectConfig[key] = value
        self.projectConfig = helper.packDict(self.projectConfig)
        
    def subKeys(self,key):
        keyList = []
        pattern = rf"__(.+?)__"
        matches = re.findall(pattern, key)
        if len(matches) > 0: 
            for match in matches:
                attrs = getattr(self,match)
                for attr in attrs:
                    keyList.append(key.replace(f"__{match}__",attr))
        else:
            keyList.append(key)
        return(keyList)

    def saveProject(self):
        output = helper.unpackDict(self.projectConfig)
        for keys,value in helper.unpackDict(self._globalConfig).items():            
            keys = self.subKeys(keys)
            for key in keys:
                if type(value) is str:
                    fpath = os.path.join(self.projectPath,key,value)
                    self.saveDict(output[key],fpath)

    def loadDict(self,file):
        out = None
        if os.path.isfile(file):
            if file.endswith('.yml'):
                with open(file) as f:
                    out = yaml.safe_load(f)
            elif file.endswith('.json'):
                with open(file) as f:
                    out = json.load(f)
        else:
            print(f"{file} does not exist")
        return(out)
    
    def saveDict(self,obj,outputPath):
        if not os.path.isdir(os.path.split(outputPath)[0]):
            os.makedirs(os.path.split(outputPath)[0])
        with open(outputPath,'w') as file:
            if outputPath.endswith('.yml'):
                yaml.safe_dump(obj,file,sort_keys=False)
            if outputPath.endswith('.json'):
                json.dump(obj,file)
    
class configFiles(myProject):
    def __init__(self,**kwds):
        super().__init__(**kwds)

class rawData(configFiles):
    def __init__(self, **kwds):
        super().__init__(**kwds)

projectPath = os.path.join(os.getcwd(),'temp2')
if os.path.isdir(projectPath):
    shutil.rmtree(projectPath)
a = rawData(projectPath=projectPath,siteID='ck')


In [ ]:
# a.save()
# a = os.makedirs('C:/temp/trashs')
# print(a)
# helper.unpackDict(a.projectConfig)
# a = {}
# if type(a) is dict and not a:
#     print('!')
a.saveProject()
b = rawData(projectPath=projectPath,siteID='ck')
c = rawData(projectPath=projectPath,siteID='pck')

In [168]:
@dataclass
class pi:
    someValue: str='cho pi'
    anotherValue: list = field(default_factory=list)
    # def __init__(self,a=None,**kwds):
    #     self.a = a
    #     super().__init__(**kwds)

    def __post_init__(self):
        for (name, field_type) in self.__annotations__.items():
            if field_type == list and not isinstance(self.__dict__[name], field_type):
                self.__dict__[name] = list([self.__dict__[name]])
            if not isinstance(self.__dict__[name], field_type):
                current_type = type(self.__dict__[name])
                raise TypeError(f"The field `{name}` was assigned by `{current_type}` instead of `{field_type}`")

        print("Check is passed successfully")
        
class cho(pi):
    def __init__(self,*args,**kwargs):
        # self.b = kwargs['b']
        print(kwargs)
        super().__init__(**kwargs)

# class baobao(cho):
#     def __init__(self,**kwds):
#         super().__init__(**kwds)
#         self.c = 'xiao'

# a = baobao(a='a',b='b')
# a.b
kwargs = {'someValue':'a','anotherValue':'b'}
yan = cho(b='x',**kwargs)


{'b': 'x', 'someValue': 'a', 'anotherValue': 'b'}


TypeError: pi.__init__() got an unexpected keyword argument 'b'

In [100]:
str(['a'])

"['a']"